In [ ]:
#from numpy.core.numeric import infty
!pip install numpy
!pip install pandas
!pip install plotly

!pip install scikit-learn # non-depreceated sklearn

Task 1

In [ ]:
import pandas as pd
# read the csv and make it a pandas.DataFrame
frame = pd.DataFrame(pd.read_csv("./online_shoppers_intention.csv"))
frame.describe(include='all') # 18 columns, including 2 categorical: Month & VisitorType, and two boolean: Weekend & Revenue

In [ ]:
#visualisation
# split the Browser 13 users from the rest
frame13 = frame.loc[frame.Browser == 13]
frameOther = frame.loc[frame.Browser != 13]

import plotly.graph_objects as go
# make a small dataframe with normalized data for plotting
def create_distribution(frameColumn):
    return pd.DataFrame({
        'Browser 13': frame13[frameColumn].value_counts(normalize=True),
        'Other browsers': frameOther[frameColumn].value_counts(normalize=True) 
    }).reset_index().rename(columns={'index': col}) 
# plot all these columns
for col in ['VisitorType', 'Region', 'Month']:
    distribution = create_distribution(col)
    fig = go.Figure()
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Browser 13'], name='Browser 13'))
    fig.add_trace(go.Bar(x=distribution[col], y=distribution['Other browsers'], name='Other browsers'))
    fig.update_layout(barmode='group', title=f'{col} Distribution')
    fig.update_yaxes(title='Count')
    fig.update_xaxes(title=f'{col}')
    fig.show()

Task 2

In [ ]:
#Prepositioning op basis van algoritmes in task 3
# Preprocess by changing all string types to an integer
def to_num(x):
    data = {
        "Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4,
        "May": 5, "June": 6, "Jul": 7, "Aug": 8,
        "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12, 
        "Returning_Visitor": 1, "Old_Visitor": 2, "New_Visitor": 3, "Other": 4, 
        True: 1, False: 0
    }
    if x not in data:
        raise ValueError(f"Invalid month string: {x}")
    return data[x]

preprocessed = (frame.copy().dropna()) # dropna() is added here for a case where the dataset would include missing values, but does nothing with the original dataset as it does not contain any
# converts strings and booleans to numbers
columns = ['Month', 'VisitorType', 'Weekend', 'Revenue']
for column in columns:
    addlist = []
    for i, j in enumerate(preprocessed[column]):
        addlist.append(to_num(j))
    preprocessed[column] = addlist

preprocessed = (preprocessed-preprocessed.min())/preprocessed.max()-preprocessed.min() # Normalization
print(preprocessed)


Task 3

In [ ]:
# affinity propagation clustering - based on documentation - VERY SLOW FOR WHOLE DATASET
from sklearn.cluster import AffinityPropagation, DBSCAN, Birch
import plotly.express as px
Data = preprocessed.copy()
# By sampling it goes faster
Sample = Data.sample(n=1000)
clustering = AffinityPropagation(random_state=2).fit(Sample)
AffinityPropagation(random_state=2)

labels = clustering.labels_
cluster_centers = clustering.cluster_centers_
n_clusters = len(cluster_centers)

print("Number of clusters:", n_clusters)
print("Cluster centers:\n", cluster_centers)


# DBSCAN clustering
# It works, but a lot of datapoints get labeled -1, meaning there is a lot of noise
# It has less clusters compared to the other methods
clustering2 = DBSCAN().fit(Data)
labels2 = clustering2.labels_
print("Labels DBSCAN", labels2[:50])   

# Birch clustering
clustering3 = Birch(n_clusters=None).fit(Data)
labels3 = clustering3.labels_
print(len(labels3), len(Data))
print("Labels Birch", labels3[:50])   
aantal_clusters = len(set(labels3))
print("Aantal clusters Birch", aantal_clusters)

fig = px.scatter(Sample, labels)
fig.show()
fig = px.scatter(Data, labels2)
fig.show()
fig = px.scatter(Data, labels3)
fig.show()


In [ ]:
#TODO visualisaties van task 3
# in apart codeblock 

Task 4

In [ ]:
# manual Silhouette score, is zo volgensmij goed. Moet alleen zoeken hoe ik deze waardes kan pakken uit de labels
import numpy as np
def Silhouette_score(point, labelSELF, labels, Data):
    clusterSELF = Data[labels == labelSELF].to_numpy()
    clusters = Data[labels != labelSELF].to_numpy()
    afstand = 0

    for points in clusterSELF:
        if set(points) == set(point):
            continue
        afstand += np.linalg.norm(point-points)
    a = afstand/len(clusterSELF)

    lowest = 999999999
    for clust in clusters:
        afstand_nearest = 0
        for points in clust:
            afstand_nearest += np.linalg.norm(point-points)
        dis = afstand_nearest/len(clust)
        if dis < lowest:
            lowest = dis
    b = lowest
    return (b-a)/(max(b,a))

def Silhouette_scores(x, labels):
    n = len(x)
    sil_values = []
    for i in range(n):
        sil_values.append(Silhouette_score(point=x.iloc[i].to_numpy(), labelSELF = labels[i], labels = labels, Data= x))
    print("Silhouette score:", sil_values)

from sklearn.metrics import davies_bouldin_score
print("Silhouette scores:")
print(f"Affinity propagation: {Silhouette_scores(Sample, labels)}")
print(f"DBSCAN: {Silhouette_scores(Data[:1000], labels2[:1000])}")
print(f"Birch Clustering: {Silhouette_scores(Data[:1000], labels3[:1000])}")
print()

# Davies Buildin Score
from sklearn.metrics import davies_bouldin_score
print("Davies Bouldin score:")
print(f"Affinity propogation: {davies_bouldin_score(Sample, labels)}")
print(f"DBSCAN: {davies_bouldin_score(Data, labels2)}")
print(f"Birch Clustering: {davies_bouldin_score(Data, labels3)}")
print()
# calinski-harabasz index
from sklearn.metrics import calinski_harabasz_score
print("Calinski Harabasz score:")
print(f"Affinity propogation: {calinski_harabasz_score(Sample, labels)}")
print(f"DBSCAN: {calinski_harabasz_score(Data, labels2)}")
print(f"Birch Clustering: {calinski_harabasz_score(Data, labels3)}")
#TODO visualize

Task 5 ALL MANUAL


In [ ]:
# dit is de basics, maar idk wat de dimensies van a en b horen te zijn
# Euclidean distance
def our_euclidean_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += (a[index] - b[index]) ** 2
    distance = distance**0.5
    return distance
# Manhattan distance
def our_manhattan_distance(a, b):
    distance = 0
    for index, _ in enumerate(a):
        distance += ((a[index] - b[index]) ** 2) ** 0.5
    return distance    
# Cosine similarity
def our_cosine_similarity(a, b):
    numerator = 0
    denominator_a = 0
    denominator_b = 0
    for index, _ in enumerate(a):
        numerator += a[index] * b[index]
        denominator_a += a[index] ** 2
        denominator_b += b[index] ** 2
    distance = numerator / (denominator_a ** 0.5 * denominator_b ** 0.5)
    return distance
# Distances effect on dbscan
clustering_c1 = DBSCAN(metric=our_euclidean_distance).fit(Data[:1000])
label_c1 = clustering_c1.labels_
print("Labels DBSCAN", label_c1)   

clustering_c2 = DBSCAN(metric=our_manhattan_distance).fit(Data[:1000])
label_c2 = clustering_c2.labels_
print("Labels DBSCAN", label_c2)   

clustering_c3 = DBSCAN(metric=our_cosine_similarity).fit(Data[:1000])
label_c3 = clustering_c3.labels_
print("Labels DBSCAN", label_c3)   

import plotly.express as px
fig = px.scatter(Data[:1000], label_c1)
fig.show()
fig = px.scatter(Data[:1000], label_c2)
fig.show()
#TODO labels zijn allemaal -1
fig = px.scatter(Data[:1000], label_c3)
fig.show()